In [1]:
'''
Run GNN policy on robot in simulation
Control inputs use (x y yaw) removed and goal rotated to body frame
Steers the robot back towards the center
'''

# load libraries
import torch
from robot_env import robot_env
import numpy as np
import pgnn_control as pgnnc
from utils import to_tensors, combine_state, wrap_to_pi, rotate, create_control_inputs

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# init env
# use the env to get the size of the inputs and outputs
# env = robot_env(show_GUI = False)
env = robot_env(show_GUI = True)
env.reset_terrain()
    
# USE_JOY = True
USE_JOY = False
if USE_JOY:
    from vrjoystick import init_joystick, read
    joy = init_joystick()
    


In [ ]:

# load the network weights
# folder = 'saved/no_tripod'
folder = 'saved/with_tripod'
PATH =  'multidesign_control_iter3.pt'
PATH = folder + PATH
# runs on CPU here
save_dict = torch.load( PATH, map_location=lambda storage, loc: storage)
urdf_names = save_dict['urdf_names']
urdf_name = 'llllll'
# urdf_name = 'lwwwnl'


gnn_state_dict= save_dict['gnn_state_dict'] 
internal_state_len = save_dict['internal_state_len'] 
message_len = save_dict['message_len'] 
hidden_layer_size = save_dict['hidden_layer_size']
goal_len =3

print(save_dict['comment'])
env.sim_speed_factor = 2
env.reset_robot(urdf_name=urdf_name, randomize_start=False)

attachments = env.attachments
modules_types = env.modules_types
print('attachments: ' + str(attachments))
print('modules_types: ' + str(modules_types))
n_modules = len(modules_types)

env_state_init = env.get_state()
module_state_len = []
for s in env_state_init:
    module_state_len.append(len(s))

state_len= np.sum(module_state_len)
action_len = env.num_joints
module_action_len = list(np.diff(env.action_indexes))

module_sa_len = module_state_len+ module_action_len

gnn_nodes = pgnnc.create_GNN_nodes(internal_state_len, 
                                   message_len, hidden_layer_size,
                                   device, goal_len=goal_len, body_input = True)
pgnnc.load_state_dicts(gnn_nodes, save_dict['gnn_state_dict']) 

# create module containers for the nodes
modules = []
for i in range(n_modules):
    modules.append(pgnnc.Module(i, gnn_nodes[modules_types[i]], device))

In [2]:
env.p.resetDebugVisualizerCamera(2.1,0,-65,[0,0,0.2],physicsClientId=env.physicsClient) 
env.reset_robot(urdf_name=urdf_name, randomize_start=False)
    
env_state = env.get_state()
last_states = [smm.to(device) for smm in to_tensors(env_state)]
    
robot_button_mapping = [0,1,2,11,3,4]
n_buttons_to_check = len(robot_button_mapping)
buttons = np.zeros(12)
vxy_scale = 1.5
vyaw_scale = 1.5
for step in range(10000):
    
    
    if USE_JOY:
        axes, buttons, povs = read(joy)
        if buttons[9]==1:
            break
        
        if np.any(buttons[0:n_buttons_to_check]):
            pressed = np.where(buttons[0:n_buttons_to_check])[0][0]
            urdf_name = urdf_names[robot_button_mapping[pressed]]
#             urdf_name = urdf_names_out[pressed]
            env.reset_terrain()
            env.reset_robot(urdf_name=urdf_name, randomize_start=False)
            attachments = env.attachments
            modules_types = env.modules_types
            n_modules = len(modules_types)
            env_state_init = env.get_state()
            module_state_len = []
            for s in env_state_init:
                module_state_len.append(len(s))
            state_len= np.sum(module_state_len)
            action_len = env.num_joints
            module_action_len = list(np.diff(env.action_indexes))
            module_sa_len = module_state_len+ module_action_len
            # create module containers for the nodes
            modules = []
            for i in range(n_modules):
                modules.append(pgnnc.Module(i, gnn_nodes[modules_types[i]], device))
            env_state = env.get_state()
            last_states = [smm.to(device) for smm in to_tensors(env_state)]
    
    chassis_yaw = env.pos_rpy[-1]
    chassis_x = env.pos_xyz[0]
    chassis_y = env.pos_xyz[1]
    
    # set direction to head
    desired_xyyaw = np.zeros(3)
    T= 20
    dt = 20./240.
    speed_scale_xy = (T*dt)*0.314*(1./0.75)
    speed_scale_yaw = (T*dt)*1.1*(1./0.75)

    desired_xyyaw[0] = speed_scale_xy
    desired_xyyaw[1] = -speed_scale_xy*chassis_y*4
    desired_xyyaw[1] = np.clip(desired_xyyaw[1], -speed_scale_xy, speed_scale_xy)
    # force to turn toward the heading
    desired_xyyaw[2] = -speed_scale_yaw*chassis_yaw/2
    desired_xyyaw[2] = np.clip(desired_xyyaw[2], -speed_scale_yaw,speed_scale_yaw)


    vect1 = np.array([desired_xyyaw[0],
            desired_xyyaw[1],
            0] )
    vect2 = np.array([np.cos(desired_xyyaw[2]),
                      np.sin(desired_xyyaw[2]), 
                      0])*np.abs(desired_xyyaw[2])


    env.draw_body_arrows([vect1*0.5/vxy_scale, 
                          0.5*vect2/vyaw_scale],
                         [[0,0,0], [0,0,1]])

    
    env_state = env.get_state()
    states = [smm.to(device) for smm in to_tensors(env_state)]
    

    goals_world = torch.tensor(desired_xyyaw, dtype=torch.float32, device=device).unsqueeze(0)
    node_inputs = create_control_inputs(last_states, goals_world)
    
    
    for module in modules: # this prevents the LSTM in the GNN nodes from 
        # learning relations over time, only over internal prop steps.
        module.reset_hidden_states(1) 

    with torch.no_grad():
        out_mean, out_var = pgnnc.run_propagations(
            modules, attachments, 2, node_inputs, device)
        u_out_mean = []
        tau_out_mean = []
        for mm in range(n_modules):
            u_out_mean.append(out_mean[mm][:,:module_action_len[mm]])
            tau_out_mean.append(out_mean[mm][:,module_action_len[mm]:])
        u_np = torch.cat(u_out_mean,-1).squeeze().numpy()
        env.step(u_np)
        
    last_states = [s.clone() for s in states]

            
    if np.dot([0,0,1], env.z_axis)<0:
        env_yaw = env.pos_rpy[-1]
        env.reset_robot(urdf_name=urdf_name, randomize_start=True)
        
    if np.sqrt(env.pos_xyz[0]**2+ env.pos_xyz[1]**2)>2:
        env_state = env.get_state()
        env_state[0][0] = -2 
        env.set_state(env_state)



NameError: name 'env' is not defined